<a href="https://colab.research.google.com/github/YBLi24/Thesis/blob/main/Conflict_Frame_Twitter.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
!nvidia-smi

Tue Apr 18 01:58:16 2023       
+-----------------------------------------------------------------------------+
| NVIDIA-SMI 525.85.12    Driver Version: 525.85.12    CUDA Version: 12.0     |
|-------------------------------+----------------------+----------------------+
| GPU  Name        Persistence-M| Bus-Id        Disp.A | Volatile Uncorr. ECC |
| Fan  Temp  Perf  Pwr:Usage/Cap|         Memory-Usage | GPU-Util  Compute M. |
|                               |                      |               MIG M. |
|===============================+======================+======================|
|   0  Tesla V100-SXM2...  Off  | 00000000:00:04.0 Off |                    0 |
| N/A   41C    P0    52W / 300W |      0MiB / 16384MiB |      0%      Default |
|                               |                      |                  N/A |
+-------------------------------+----------------------+----------------------+
                                                                               
+-------

In [ ]:
! pip install transformers

In [ ]:
import pandas as pd
import numpy as np
import torch
from torch.utils.data import Dataset
import numpy as np
from torch import nn
from torch.utils.data import DataLoader
from transformers import BertTokenizer, BertForSequenceClassification
from sklearn.metrics import f1_score

In [ ]:
df = pd.read_csv ("/content/drive/MyDrive/【Thesis】/twitter_500_preprocessed.csv")
labels = {0:0, 1:1, 2:1, 3:1, 4:1}
df["cf_binary"]=[labels[label]for label in df["Conflict frames"]]
df

In [ ]:
df_train, df_val = np.split(df.sample(frac=1, random_state=42), [int(.6*len(df))])
print(len(df_train), len(df_val))

In [ ]:
class TextLabelDataset(Dataset):
    def __init__(self, dataframe, text_column, label_column):
        self.dataframe = dataframe
        self.text_column = text_column
        self.label_column = label_column

    def __len__(self):
        return len(self.dataframe)

    def __getitem__(self, idx):
        if torch.is_tensor(idx):
            idx = idx.tolist()

        text = self.dataframe.iloc[idx][self.text_column]
        label = self.dataframe.iloc[idx][self.label_column]

        return text, torch.tensor(label, dtype=torch.long)

In [ ]:
train_dataset = TextLabelDataset(df_train, "text_preprocessed", "cf_binary")
val_dataset = TextLabelDataset(df_val, "text_preprocessed", "cf_binary")

In [ ]:
def train_bert_binary_classifier(train_dataset, val_dataset, epochs=10, batch_size=16, learning_rate=3e-6, device="cuda"):
    tokenizer = BertTokenizer.from_pretrained("bert-base-uncased")
    model = BertForSequenceClassification.from_pretrained("bert-base-uncased", num_labels=2)
    model.to(device)
    
    train_loader = DataLoader(train_dataset, batch_size=batch_size, shuffle=True)
    val_loader = DataLoader(val_dataset, batch_size=batch_size, shuffle=False)
    optimizer = torch.optim.AdamW(model.parameters(), lr=learning_rate)
    loss_function = nn.CrossEntropyLoss()

    best_val_loss = float("inf")
    best_model = None
    
    for epoch in range(epochs):
        model.train()
        train_loss = 0
        train_correct = 0
        train_total = 0
        for batch in train_loader:
            optimizer.zero_grad()
            inputs, labels = batch
            inputs = tokenizer(inputs, padding = "max_length", max_length = 512, return_tensors="pt", truncation=True).to(device)
            labels = labels.to(device)
            
            outputs = model(**inputs).logits
            loss = loss_function(outputs, labels)
            loss.backward()
            optimizer.step()

            train_loss += loss.item()
            train_total += labels.size(0)
            train_correct += (outputs.argmax(1) == labels).sum().item()
        
        train_loss /= len(train_loader)
        train_accuracy = train_correct / train_total

        model.eval()
        val_loss = 0
        val_correct = 0
        val_total = 0
        val_preds = []
        val_true = []
        with torch.no_grad():
            for batch in val_loader:
                inputs, labels = batch
                inputs = tokenizer(inputs, padding = "max_length", max_length = 512, return_tensors="pt", truncation=True).to(device)
                labels = labels.to(device)
                
                outputs = model(**inputs).logits
                loss = loss_function(outputs, labels)
                val_loss += loss.item()
                
                val_total += labels.size(0)
                val_correct += (outputs.argmax(1) == labels).sum().item()
                val_preds.extend(outputs.argmax(1).cpu().numpy())
                val_true.extend(labels.cpu().numpy())

        val_loss /= len(val_loader)
        val_accuracy = val_correct / val_total
        val_f1_score = f1_score(val_true, val_preds)
        #print(val_preds)
        #print(val_true)
            
        print(f'Epoch: {epoch + 1} '
              f'| Train Loss: {train_loss:.3f} '
              f'| Train Accuracy: {train_accuracy:.3f} '
              f'| Val Loss: {val_loss:.3f} '
              f'| Val Accuracy: {val_accuracy:.3f} '
              f'| Val F1 Score: {val_f1_score:.3f}')

        if val_loss < best_val_loss:
            best_val_loss = val_loss
            best_model = model.state_dict()

    model.load_state_dict(best_model)
    return model, best_model

In [ ]:
trained_model, best_model_state_dict = train_bert_binary_classifier(train_dataset, val_dataset)

In [ ]:
import numpy as np
import pandas as pd
from sklearn.feature_extraction.text import CountVectorizer, TfidfTransformer
from sklearn.svm import SVC
from sklearn.pipeline import Pipeline
from sklearn.model_selection import train_test_split
from sklearn.metrics import accuracy_score, classification_report, confusion_matrix

# Load your data (replace with your dataset)
# data = pd.read_csv('path_to_your_data.csv')
X = df["text_preprocessed"]
y = df["cf_binary"]

# Split the data into training and testing sets
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.3, random_state=42)

# Build the SVM pipeline
svm_pipeline = Pipeline([
    ('vect', CountVectorizer()),
    ('tfidf', TfidfTransformer()),
    ('clf', SVC(kernel='linear', probability=True)),
])

# Train the SVM classifier
svm_pipeline.fit(X_train, y_train)

# Test the SVM classifier
svm_y_pred = svm_pipeline.predict(X_test)
svm_y_prob = svm_pipeline.predict_proba(X_test)

# Print classification metrics for SVM
print(classification_report(y_test, svm_y_pred))
print(confusion_matrix(y_test, svm_y_pred))
print("Accuracy:", accuracy_score(y_test, svm_y_pred))
print("F1:", f1_score(y_test, svm_y_pred))
